In [9]:
!pip install qiskit torch numpy pandas scikit-learn

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit
#ParameterVector was redudant
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

#To control the random trainable parameters during development:
torch.manual_seed(42)

#Prep Training Data (Breast Cancer Wisconsin)

#Load the data file
df = pd.read_csv("wdbc.data", header=None)

#Assign column names
columns = ['id', 'diagnosis'] + [f'feature_{i}' for i in range(1, 31)]
df.columns = columns

#Drop the ID column
df = df.drop(columns=['id'])

#Encode diagnosis: M = 1, B = 0
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

#Convert to numpy arrays
X = df.drop(columns=['diagnosis']).values.astype(np.float32)
Y = df['diagnosis'].values.astype(np.float32).reshape(-1, 1)

#Normalize features manually (z-score)
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

#Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

#Manual train/test split (80/20)
num_samples = X_tensor.shape[0]
indices = torch.randperm(num_samples)

split_idx = int(num_samples * 0.8)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]

X_train = X_tensor[train_indices]
Y_train = Y_tensor[train_indices]
X_test = X_tensor[test_indices]
Y_test = Y_tensor[test_indices]

#VQA Circuit
n_qubits = 4
#Params were redudant

def create_vqa_circuit(input_data, weights):
    qc = QuantumCircuit(n_qubits)
    shift = 0  # index into weights array

    # 1) Triple data re‑uploading with CZ entanglement
    for _ in range(3):
        for i in range(n_qubits):
            qc.ry(input_data[i % len(input_data)], i)
        for i in range(n_qubits - 1):
            qc.cz(i, i + 1)
        qc.barrier()

    # 2) Two variational layers: mixed-axis rotations + ring CNOTs
    for _ in range(2):
        # each layer has 2 rotations per qubit
        for i in range(n_qubits):
            qc.rx(weights[shift], i); shift += 1
            qc.ry(weights[shift], i); shift += 1
        # ring entanglement
        for i in range(n_qubits):
            qc.cx(i, (i + 1) % n_qubits)
        qc.barrier()

    # 3) Final fine‑tune layer: one Ry per qubit
    for i in range(n_qubits):
        qc.ry(weights[shift], i)
        shift += 1

    return qc


#Qiskit StatevectorEstimator primitive
estimator = StatevectorEstimator()
#observables = [ SparsePauliOp("Z" + "I" * (n_qubits-1)) ] #Single multi-qubit gate
observables = [
    SparsePauliOp("".join("Z" if j == i else "I" for j in range(n_qubits)))
    for i in range(n_qubits)
] #A weighted sum will be applied using trainable reeadout parameters

#PyTorch Custom Autograd Function For VQA Layer
class VQALayerFunction(Function):
    @staticmethod
    def forward(ctx, input_tensor, weights):
        input_vals = input_tensor.detach().numpy()
        weight_vals = weights.detach().numpy()
        ctx.save_for_backward(input_tensor, weights)

        qc = create_vqa_circuit(input_vals, weight_vals)
        job = estimator.run([(qc, observables)])
        expval = job.result()[0].data.evs

        return torch.tensor([expval], dtype=torch.float32)

    @staticmethod
    def backward(ctx, grad_output):
        input_tensor, weights = ctx.saved_tensors
        input_vals = input_tensor.detach().numpy()
        weight_vals = weights.detach().numpy()
        shift = np.pi / 2
        grads = []

        for i in range(len(weight_vals)): # Grad is calculated using parameter shift rule
            #print("Make changes here")
            #w abbreviates weight_vals
            w_plus, w_minus = weight_vals.copy(), weight_vals.copy()
            w_plus[i]  += shift
            w_minus[i] -= shift

            circuit_plus  = create_vqa_circuit(input_vals, w_plus)
            circuit_minus = create_vqa_circuit(input_vals, w_minus)

            EV_plus  = estimator.run([(circuit_plus, observables)]).result()[0].data.evs[0]
            EV_minus = estimator.run([(circuit_minus, observables)]).result()[0].data.evs[0]

            grads.append( 0.5 * (EV_plus - EV_minus) )


        grads_tensor = torch.tensor(grads, dtype=torch.float32)
        return None, (grad_output.view(-1)[0] * grads_tensor).view(-1)

#Quantum Layer as PyTorch Module
class VQALayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(5 * n_qubits)) #trainable parameters for the circuit
        self.best_weights = self.weights.detach().clone() #to capture the best weights during training
        self.readout_w = nn.Parameter(torch.ones(n_qubits) / n_qubits) #trainable parameters for the a weighted Pauli sum observable

    def forward(self, x):
        #run quantum circuit on each sample
        expvals = torch.stack([
            VQALayerFunction.apply(x[i], self.weights)
            for i in range(x.size(0))
        ], dim=0).squeeze(1)  #tensor shape [batch, n_qubits]

        return expvals #Returns expectation for each qubit

        #return torch.stack([VQALayerFunction.apply(x[i], self.weights) for i in range(x.size(0))]).view(-1, 1)

#Full Hybrid Model
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.classical = nn.Linear(X_tensor.shape[1], n_qubits) #Classic preprocessing layer
        self.quantum = VQALayer() #VQA layer
        self.readout_w = nn.Parameter(torch.ones(n_qubits))
        #self.output = nn.Linear(n_qubits, 1) #Final classical layer

    def forward(self, x):
        x = self.classical(x)
        x = torch.tanh(x)  #Activation before quantum layer
        x = self.quantum(x)

        x = (x * self.readout_w).sum(dim=1, keepdim=True)
        #x = self.output(x)

        return torch.sigmoid(x)

model = HybridModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.04)
loss_fn = nn.BCELoss()

#Training Loop
batch_size = 16
train_dataset = TensorDataset(X_train, Y_train)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

best_rocauc = 0

for epoch in range(8):
    total_loss = 0.0

    #Iterate over batches
    for Xb, Yb in train_loader:
        optimizer.zero_grad()
        preds = model(Xb)
        loss = loss_fn(preds, Yb)
        loss.backward()

        #Print gradient norms for all parameters
        # for name, param in model.named_parameters():
        #     if param.grad is not None:
        #         print(f"{name} grad_norm = {param.grad.norm():.4f}")

        optimizer.step()
        total_loss += loss.item()

    #After all batches, compute epoch‑level metrics on the full training set
    with torch.no_grad():
        preds_full = model(X_train)
        acc_full   = ((preds_full > 0.5).float() == Y_train).float().mean()
        y_scores   = preds_full.view(-1).cpu().numpy()
        y_true     = Y_train.view(-1).cpu().numpy().astype(int)

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Avg Loss: {avg_loss:.4f} | Accuracy: {acc_full.item()*100:.2f}%")

    #Record best params
    rocauc = roc_auc_score(y_true, y_scores)

    if rocauc > best_rocauc:
        model.quantum.best_weights = model.quantum.weights.detach().clone()
        best_rocauc = rocauc

    if epoch == 7:
        print(f"Best weights: {model.quantum.best_weights}\n Best ROCAUC: {best_rocauc}")

#Implement Testing Loop:
#print("Make changes here")

#TO SOLVE THE ERROR IN TESTING WITHOUT RETRAINING, THE TESTING CODE WAS SEPERATED TO PERSERVE THIS CELL'S OUTPUTS

Epoch 1 | Avg Loss: 0.6268 | Accuracy: 83.74%
Epoch 2 | Avg Loss: 0.3541 | Accuracy: 92.53%
Epoch 3 | Avg Loss: 0.2396 | Accuracy: 93.63%
Epoch 4 | Avg Loss: 0.1946 | Accuracy: 94.29%
Epoch 5 | Avg Loss: 0.1946 | Accuracy: 94.95%
Epoch 6 | Avg Loss: 0.1704 | Accuracy: 94.73%
Epoch 7 | Avg Loss: 0.1661 | Accuracy: 95.60%
Epoch 8 | Avg Loss: 0.1562 | Accuracy: 94.95%
Best weights: tensor([ 1.6283, -0.0642, -0.0969,  0.4047, -0.7979,  0.8536,  0.3498,  3.4695,
         0.1063, -0.4137, -1.7513,  0.7449,  1.2030, -0.9913, -1.3877, -0.3289,
         1.4306, -0.2583, -0.7917, -0.0676])
 Best ROCAUC: 0.986305081919373


TypeError: cannot assign 'torch.FloatTensor' as parameter 'weights' (torch.nn.Parameter or None expected)

In [24]:
#Implement Testing Loop:
#print("Make changes here")
#Load best params at the end
model.quantum.weights.data.copy_(model.quantum.best_weights.detach().clone())

model.eval()
with torch.no_grad():
    #Forward pass on test set
    preds_test = model(X_test).view(-1).cpu().numpy()
    y_true = Y_test.view(-1).cpu().numpy().astype(int)
    y_pred_labels = (preds_test > 0.5).astype(int)

    #Compute metrics
    acc    = accuracy_score(y_true, y_pred_labels)
    prec   = precision_score(y_true, y_pred_labels)
    rec    = recall_score(y_true, y_pred_labels)
    f1     = f1_score(y_true, y_pred_labels)
    rocauc = roc_auc_score(y_true, preds_test)
    cm     = confusion_matrix(y_true, y_pred_labels)

#Print results
print(f"Test Accuracy:  {acc:.4f}")
print(f"Precision:      {prec:.4f}")
print(f"Recall:         {rec:.4f}")
print(f"F1-Score:       {f1:.4f}")
print(f"ROC AUC:        {rocauc:.4f}")
print("Confusion Matrix:")
print(cm)

Test Accuracy:  0.8860
Precision:      0.8511
Recall:         0.8696
F1-Score:       0.8602
ROC AUC:        0.9680
Confusion Matrix:
[[61  7]
 [ 6 40]]


In [25]:
#Save just the best weights tensor
torch.save(model.quantum.best_weights, "quantum_best_weights.pt")

#To load it back:
#loaded_weights = torch.load("quantum_best_weights.pt")
#and assign it via:
#model.quantum.weights.data.copy_(loaded_weights)

In [26]:
torch.save({
    "model_state_dict": model.state_dict(),
    "best_quantum_weights": model.quantum.best_weights
}, "full_model_checkpoint.pt")

#Load with:
#checkpoint = torch.load("full_model_checkpoint.pt")
#model.load_state_dict(checkpoint["model_state_dict"])
#model.quantum.weights.data.copy_(checkpoint["best_quantum_weights"])
